##**Etiquetado de registros de trabajos con y sin tareas pendientes**

**Preparar notebook**

In [ ]:
## Conectar el notebook a googledrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Importar librerias necesarias
import nltk
import pandas as pd
import numpy as np
import re
import codecs
import matplotlib.pyplot as plt
from collections import Counter
from nltk.util import ngrams
from nltk import bigrams
import spacy

**Carga de los Datos y Creación del *Data Set* para el análisis**

In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/ProyectoIntegrador/Datos/datos_preparados.csv', encoding='utf-8')
#Habilitar la siguiente línea para correr el proyecto de manera local
#df2 = pd.read_csv('/content/datos_preparados.csv', encoding='utf-8')
print(df2.shape)
df2.head()

(156792, 7)


,Unnamed: 0,RESUMEN,WORKLOGID,DETALLE,procesado,tokens,tokens_proc
0,0,CIERREOT,2754950,Se revisa y se encuentra poste de 12 MTS de ji...,se revisa y se encuentra poste de metros de ji...,"['se', 'revisa', 'y', 'se', 'encuentra', 'post...","['revisa', 'poste', 'reventado', 'vehiculo', '..."
1,1,CIERREOT,3005990,Localizar falla circuito primario sub guayabal...,localizar falla circuito primario subestacion ...,"['localizar', 'falla', 'circuito', 'primario',...","['localizar', 'falla', 'circuito', 'primario',..."
2,2,CIERREOT,2634320,Se retiraron 5 ramas de Palma que estaban sobr...,se retiraron arbol de palma que estaban sobre ...,"['se', 'retiraron', 'arbol', 'de', 'palma', 'q...","['retiraron', 'arbol', 'palma', 'aisladero', '..."
3,3,CIERREOT,2634321,R27-07 se visita Cll57c#34-36 revisión por cal...,r se visita cll c revision por calidad del ser...,"['r', 'se', 'visita', 'cll', 'c', 'revision', ...","['cll', 'revision', 'calidad', 'inspecciona', ..."
4,4,CIERREOT,2742512,Con el evento Mar 4879178 en carrera 33 por 29...,con el evento mar en carrera por c del r donde...,"['con', 'el', 'evento', 'mar', 'en', 'carrera'...","['mar', 'carrera', 'trabajador', 'reparan', 'l..."


In [ ]:
df2.columns

Index(['Unnamed: 0', 'RESUMEN', 'WORKLOGID', 'DETALLE', 'procesado', 'tokens',
       'tokens_proc'],
      dtype='object')

In [ ]:
## Seleccionar columnas de interés
df2 = df2[['WORKLOGID', 'procesado', 'tokens_proc']]
df2.shape

(156792, 3)

In [ ]:
## Recuperar los tokes (como lista) que se extrajeron en el textprep y que al exportar el archivo .csv se cargaron como cadenas de texto
df2['tokens_proc'] = df2['tokens_proc'].apply(lambda x: re.sub('[\[\]\']+', '', str(x)))
df2['tokens_proc'] = df2['tokens_proc'].apply(lambda x: x.split(', '))

In [ ]:
df2.head()

,WORKLOGID,procesado,tokens_proc
0,2754950,se revisa y se encuentra poste de metros de ji...,"[revisa, poste, reventado, vehiculo, recoje, e..."
1,3005990,localizar falla circuito primario subestacion ...,"[localizar, falla, circuito, primario, subesta..."
2,2634320,se retiraron arbol de palma que estaban sobre ...,"[retiraron, arbol, palma, aisladero, primario,..."
3,2634321,r se visita cll c revision por calidad del ser...,"[cll, revision, calidad, inspecciona, evidenci..."
4,2742512,con el evento mar en carrera por c del r donde...,"[mar, carrera, trabajador, reparan, linea, pri..."


In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/ProyectoIntegrador/Datos/export.csv', encoding='latin-1')
#Habilitar la siguiente línea para correr el proyecto de manera local
#df1 = pd.read_csv('/content/export.csv', encoding='latin-1')
print(df1.shape)
df1.head(2)

(259762, 15)


,REPORTDATE,WONUM,DESCRIPTION,STATUS,WORKTYPE,ASSETNUM,LOCATION,REGION,ANCESTOR,SOURCESYSID,FAILURECODE,PROBLEMCODE,RESUMEN,WORKLOGID,DETALLE
0,29/05/2022 02:26:00,10016825,Atención Evento M_EPM_EN_4885195,CERRADA,CPROG,NaN,32427-4,METROPOLITANA,R05-14,MAR,POSTE,CAIDO,CIERREOT,2754950,Se revisa y se encuentra poste de 12 MTS de ji...
1,01/09/2022 02:56:24,12523772,Reparar líneas primaria subterránea fallada,CERRADA,CPROG,3206258,32427-4,METROPOLITANA,R05-14,MAR,NaN,NaN,CIERREOT,3005990,Localizar falla circuito primario sub guayabal...


In [ ]:
## Seleccionar registros asociados con tipos de trabajo de interés (Trabajos correctivos)
df1 = df1[df1['WORKTYPE'].isin(['CINME', 'CPROG'])]

In [ ]:
## Seleccionar columnas de interés
df1 = df1[['REPORTDATE','WORKTYPE', 'REGION', 'ANCESTOR', 'FAILURECODE','PROBLEMCODE','WORKLOGID']].reset_index(drop=True)
print(df1.shape)
df1.head(5)

(158751, 7)


,REPORTDATE,WORKTYPE,REGION,ANCESTOR,FAILURECODE,PROBLEMCODE,WORKLOGID
0,29/05/2022 02:26:00,CPROG,METROPOLITANA,R05-14,POSTE,CAIDO,2754950
1,01/09/2022 02:56:24,CPROG,METROPOLITANA,R05-14,NaN,NaN,3005990
2,12/04/2022 09:43:00,CINME,METROPOLITANA,R15-20,RED.PRI,FALL.LIN,2634320
3,12/04/2022 12:32:00,CINME,METROPOLITANA,R15-20,RED.SEC,FALL.LIN,2634321
4,24/05/2022 17:48:00,CINME,METROPOLITANA,R15-20,POSTE,CAIDO,2742512


In [ ]:
df = df2.merge(df1, how = 'inner', on='WORKLOGID')
print(df.shape)
df.head()

(156792, 9)


,WORKLOGID,procesado,tokens_proc,REPORTDATE,WORKTYPE,REGION,ANCESTOR,FAILURECODE,PROBLEMCODE
0,2754950,se revisa y se encuentra poste de metros de ji...,"[revisa, poste, reventado, vehiculo, recoje, e...",29/05/2022 02:26:00,CPROG,METROPOLITANA,R05-14,POSTE,CAIDO
1,3005990,localizar falla circuito primario subestacion ...,"[localizar, falla, circuito, primario, subesta...",01/09/2022 02:56:24,CPROG,METROPOLITANA,R05-14,NaN,NaN
2,2634320,se retiraron arbol de palma que estaban sobre ...,"[retiraron, arbol, palma, aisladero, primario,...",12/04/2022 09:43:00,CINME,METROPOLITANA,R15-20,RED.PRI,FALL.LIN
3,2634321,r se visita cll c revision por calidad del ser...,"[cll, revision, calidad, inspecciona, evidenci...",12/04/2022 12:32:00,CINME,METROPOLITANA,R15-20,RED.SEC,FALL.LIN
4,2742512,con el evento mar en carrera por c del r donde...,"[mar, carrera, trabajador, reparan, linea, pri...",24/05/2022 17:48:00,CINME,METROPOLITANA,R15-20,POSTE,CAIDO


In [ ]:
## Renombrar columnas para facilitar comprensión
columnas = {'REPORTDATE':'FECHA_REGISTRO','WORKTYPE':'TIPO_TRABAJO', 'ANCESTOR':'CIRCUITO', 'FAILURECODE':'COD_FALLA', 'PROBLEMCODE':'COD_PROBLEMA'}
df.rename(columns=columnas, inplace = True)
df.head(2)

,WORKLOGID,procesado,tokens_proc,FECHA_REGISTRO,TIPO_TRABAJO,REGION,CIRCUITO,COD_FALLA,COD_PROBLEMA
0,2754950,se revisa y se encuentra poste de metros de ji...,"[revisa, poste, reventado, vehiculo, recoje, e...",29/05/2022 02:26:00,CPROG,METROPOLITANA,R05-14,POSTE,CAIDO
1,3005990,localizar falla circuito primario subestacion ...,"[localizar, falla, circuito, primario, subesta...",01/09/2022 02:56:24,CPROG,METROPOLITANA,R05-14,NaN,NaN


In [ ]:
### función para unir los tokens y poder aplicar findall##
def unir_tokens (text):
  doc = ' '.join(text)
  return doc

In [ ]:
##aplicar la función de unir tokens##
df['tokensunidos']=df['tokens_proc'].apply(lambda x: unir_tokens(x))
df.head()

,WORKLOGID,procesado,tokens_proc,FECHA_REGISTRO,TIPO_TRABAJO,REGION,CIRCUITO,COD_FALLA,COD_PROBLEMA,tokensunidos
0,2754950,se revisa y se encuentra poste de metros de ji...,"[revisa, poste, reventado, vehiculo, recoje, e...",29/05/2022 02:26:00,CPROG,METROPOLITANA,R05-14,POSTE,CAIDO,revisa poste reventado vehiculo recoje estruct...
1,3005990,localizar falla circuito primario subestacion ...,"[localizar, falla, circuito, primario, subesta...",01/09/2022 02:56:24,CPROG,METROPOLITANA,R05-14,NaN,NaN,localizar falla circuito primario subestacion ...
2,2634320,se retiraron arbol de palma que estaban sobre ...,"[retiraron, arbol, palma, aisladero, primario,...",12/04/2022 09:43:00,CINME,METROPOLITANA,R15-20,RED.PRI,FALL.LIN,retiraron arbol palma aisladero primario urgen...
3,2634321,r se visita cll c revision por calidad del ser...,"[cll, revision, calidad, inspecciona, evidenci...",12/04/2022 12:32:00,CINME,METROPOLITANA,R15-20,RED.SEC,FALL.LIN,cll revision calidad inspecciona evidencia arb...
4,2742512,con el evento mar en carrera por c del r donde...,"[mar, carrera, trabajador, reparan, linea, pri...",24/05/2022 17:48:00,CINME,METROPOLITANA,R15-20,POSTE,CAIDO,mar carrera trabajador reparan linea primarias...


In [ ]:
## Obtener listado de tokens
tokens1 = []
for t in df['tokens_proc']:
    tokens1.extend(t)
print('Cantidad total de tokens = ',str(len(tokens1)))
## Distribución de frecuencia de los tokens. Tamaño del BoW
fdist_tokens = nltk.FreqDist(tokens1)
print('Tamanno del BoW=',len(fdist_tokens))

Cantidad total de tokens =  1671471
Tamanno del BoW= 11117


In [ ]:
## Distribución de frecuencia de los bigramas.
fdist_bigrams = nltk.FreqDist(bigrams(tokens1))
print('Tamanno del listado de bigramas=',len(fdist_bigrams))

Tamanno del listado de bigramas= 417158


In [ ]:
#Listado de "n" bigramas más comunes
top_bigrams = fdist_bigrams.most_common(20)
top_bigrams

[(('quedo', 'normal'), 24644),
 (('fusible', 'transformador'), 14678),
 (('quedo', 'pendiente'), 9121),
 (('transformador', 'quedo'), 7046),
 (('red', 'secundaria'), 5368),
 (('poste', 'secundario'), 4831),
 (('normal', 'fusible'), 4754),
 (('transformador', 'fusible'), 4290),
 (('aguas', 'abajo'), 4179),
 (('fusible', 'quedo'), 3517),
 (('afecto', 'transformador'), 3429),
 (('arbol', 'linea'), 3348),
 (('fusible', 'quemado'), 3321),
 (('quedo', 'normalizado'), 3269),
 (('linea', 'secundaria'), 3030),
 (('fusible', 'aisladero'), 2752),
 (('repara', 'acometida'), 2688),
 (('poste', 'primario'), 2604),
 (('verifica', 'quedo'), 2564),
 (('linea', 'primaria'), 2471)]

**Identificación de registros de trabajo con tareas pendientes**

In [ ]:
##buscar la palabra "quedo" y luego "pendiente"##
def next_word(sentence):
  result = re.findall(r'quedo.*', sentence)
  if (len(result) > 0):
    u= ''.join(map(str, result))
    temp= re.findall(r'pendiente.*', u)
    if (len(temp) > 0):
      return(temp[0])
    else:
      return('NA')
  else:
    return('NA')

In [ ]:
##aplicar la función de traer las filas con "quedo" "pendiente" y las palabras siguientes##
df['pendiente']=df['tokensunidos'].apply(lambda x: next_word(x))
df

,WORKLOGID,procesado,tokens_proc,FECHA_REGISTRO,TIPO_TRABAJO,REGION,CIRCUITO,COD_FALLA,COD_PROBLEMA,tokensunidos,pendiente
0,2754950,se revisa y se encuentra poste de metros de ji...,"[revisa, poste, reventado, vehiculo, recoje, e...",29/05/2022 02:26:00,CPROG,METROPOLITANA,R05-14,POSTE,CAIDO,revisa poste reventado vehiculo recoje estruct...,NA
1,3005990,localizar falla circuito primario subestacion ...,"[localizar, falla, circuito, primario, subesta...",01/09/2022 02:56:24,CPROG,METROPOLITANA,R05-14,NaN,NaN,localizar falla circuito primario subestacion ...,NA
2,2634320,se retiraron arbol de palma que estaban sobre ...,"[retiraron, arbol, palma, aisladero, primario,...",12/04/2022 09:43:00,CINME,METROPOLITANA,R15-20,RED.PRI,FALL.LIN,retiraron arbol palma aisladero primario urgen...,NA
3,2634321,r se visita cll c revision por calidad del ser...,"[cll, revision, calidad, inspecciona, evidenci...",12/04/2022 12:32:00,CINME,METROPOLITANA,R15-20,RED.SEC,FALL.LIN,cll revision calidad inspecciona evidencia arb...,NA
4,2742512,con el evento mar en carrera por c del r donde...,"[mar, carrera, trabajador, reparan, linea, pri...",24/05/2022 17:48:00,CINME,METROPOLITANA,R15-20,POSTE,CAIDO,mar carrera trabajador reparan linea primarias...,NA
...,...,...,...,...,...,...,...,...,...,...,...
156787,2927475,se normalizo aisladero sin marcar fase s para ...,"[normalizo, aisladero, marcar, fase, solo, tra...",06/08/2022 07:01:00,CINME,OCCIDENTE,601-11,RED.PRI,FALL.LIN,normalizo aisladero marcar fase solo transform...,NA
156788,3157566,se visita cr ed metropolitan en legalizacion e...,"[legalizacion, ayer, solo, datos, asi, entrar,...",19/10/2022 09:02:00,CINME,METROPOLITANA,R15-22,RED.SEC,FALL.LIN,legalizacion ayer solo datos asi entrar sistem...,NA
156789,3056493,se le presto a pollo al personal del cld para ...,"[presto, pollo, cld, abrir, circuitos, trabajo]",20/09/2022 07:46:00,CINME,SUROESTE,519-14,RED.PRI,FALL.LIN,presto pollo cld abrir circuitos trabajo,NA
156790,3055378,se aislo el reconectador r y se abrieron las c...,"[aislo, reconectador, abrieron, cuchillas, sal...",20/09/2022 07:47:00,CINME,SUROESTE,519-14,RED.PRI,FALL.LIN,aislo reconectador abrieron cuchillas salida t...,NA


* Exportar el conjunto de datos etiquetados de registros de trabajo con y sin tareas pendientes

In [ ]:
df.to_csv('/content/drive/MyDrive/ProyectoIntegrador/Datos/datos_pendiente_Nopendiente.csv')
#Habilitar la siguiente línea para correr el proyecto de manera local
#df.to_csv('/content/datos_pendiente_Nopendiente.csv')

In [ ]:
df2=df[df['pendiente']!='NA']

In [ ]:
df2

,WORKLOGID,procesado,tokens_proc,FECHA_REGISTRO,TIPO_TRABAJO,REGION,CIRCUITO,COD_FALLA,COD_PROBLEMA,tokensunidos,pendiente
16,2547376,se repara trenza rota y se cambia conector en ...,"[repara, trenza, rota, conector, barra, secund...",28/02/2022 17:27:43,CPROG,NORTE,304-11,NaN,NaN,repara trenza rota conector barra secundaria t...,pendientes transformador afecto
24,2457599,se visita el sitio del reporte y se encuentra ...,"[reporte, transformador, amarre, fusible, quem...",30/01/2022 07:18:00,CINME,OCCIDENTE,602-12,RED.SEC,FALL.LIN,reporte transformador amarre fusible quemado f...,pendientes abeja
25,2789392,se realizo poda en varios ramales secundarios ...,"[poda, varios, ramales, secundarios, fusible, ...",11/06/2022 19:14:00,CINME,OCCIDENTE,602-12,RED.PRI,FALL.LIN,poda varios ramales secundarios fusible transf...,pendiente cuadrilla poda debe realizar poda gr...
27,2857541,se encuentra linea primarias de en el suelo se...,"[linea, primarias, suelo, procede, repara, dan...",11/07/2022 09:02:00,CINME,OCCIDENTE,602-12,RED.PRI,FALL.HER,linea primarias suelo procede repara dano caus...,pendiente intervenir dos poste mas verda aldea
30,2437852,se visita la ruta y se inspeccionan dos poste ...,"[ruta, inspeccionan, dos, poste, reportados, c...",21/01/2022 09:15:00,CPROG,OCCIDENTE,602-12,POSTE,STD,ruta inspeccionan dos poste reportados cuales ...,pendiente doble terminal viento terminal vient...
...,...,...,...,...,...,...,...,...,...,...,...
156704,3108241,se incuentra r rechazado se abre transversal e...,"[incuentra, rechazado, abre, transversal, pone...",08/10/2022 09:51:00,CPROG,NORTE,306-13,POSTE,LADEADO,incuentra rechazado abre transversal ponen cie...,pendiente enderezar poste cambiarlo podar arbo...
156712,3038792,se reviso se trata de un puente con sercania a...,"[reviso, trata, puente, sercania, estructura, ...",14/09/2022 01:55:00,CINME,METROPOLITANA,R03-09,RED.PRI,FALL.LIN,reviso trata puente sercania estructura separo...,pendiente cable ecologico linea estan construy...
156741,2943207,se verifica tarea en terreno y se encuentra pu...,"[verifica, puente, primario, roto, asignar, oj...",11/08/2022 09:21:00,CINME,NORDESTE,110-12,RED.PRI,FALL.LIN,verifica puente primario roto asignar ojala tr...,pendiente riesgo
156761,3070258,se coloco conector cruceta en el primer apoyo ...,"[coloco, conector, cruceta, primer, apoyo, agu...",24/09/2022 21:01:00,CINME,NORDESTE,101-11,RED.PRI,FALL.LIN,coloco conector cruceta primer apoyo aguas aba...,pendiente caja primaria transformador quedo


In [ ]:
df2.shape

(10468, 11)

* Exportar los datos etiquetados con pendientes para ser usados en el análisis de k-means

In [ ]:
df2.to_csv('/content/drive/MyDrive/ProyectoIntegrador/Datos/datos_pendiente.csv')
#Habilitar la siguiente línea para correr el proyecto de manera local
#df2.to_csv('/content/datos_pendiente.csv')

* Identificación de bigramas comunes entre los registros con pendientes

In [ ]:
## Obtener listado de tokens
tokens_pend = []
for t in df2['tokens_proc']:
    tokens_pend.extend(t)
print('Cantidad total de tokens = ',str(len(tokens_pend)))
## Distribución de frecuencia de los tokens. Tamaño del BoW
fdist_tokens = nltk.FreqDist(tokens_pend)
print('Tamanno del BoW=',len(fdist_tokens))

Cantidad total de tokens =  195145
Tamanno del BoW= 7273


In [ ]:
## Distribución de frecuencia de los bigramas.
fdist_bigrams_pend = nltk.FreqDist(bigrams(tokens_pend))
print('Tamanno del listado de bigramas=',len(fdist_bigrams))

Tamanno del listado de bigramas= 417158


In [ ]:
#Listado de "n" bigramas más comunes
top_bigrams_pend = fdist_bigrams_pend.most_common(20)
top_bigrams_pend

[(('quedo', 'pendiente'), 9107),
 (('transformador', 'quedo'), 1109),
 (('pendiente', 'repara'), 1044),
 (('aguas', 'abajo'), 916),
 (('quedo', 'normal'), 873),
 (('poste', 'secundario'), 847),
 (('fusible', 'transformador'), 760),
 (('red', 'secundaria'), 656),
 (('pendiente', 'reparar'), 626),
 (('poste', 'primario'), 553),
 (('pendiente', 'poste'), 521),
 (('normal', 'quedo'), 518),
 (('dps', 'primario'), 444),
 (('pendiente', 'poda'), 442),
 (('cuadrilla', 'poda'), 431),
 (('pendiente', 'cuadrilla'), 429),
 (('linea', 'primaria'), 428),
 (('caja', 'primaria'), 425),
 (('linea', 'secundaria'), 416),
 (('arbol', 'linea'), 391)]

In [ ]:
## Distribución de frecuencia de los trigramas.
fdist_trigrams_pend = nltk.FreqDist(ngrams(tokens_pend,3))
print('Tamanno del listado de trigramas=',len(fdist_trigrams_pend))

Tamanno del listado de trigramas= 134717


In [ ]:
#Listado de "n" trigramas más comunes
top_trigrams_pend = fdist_trigrams_pend.most_common(20)
top_trigrams_pend

[(('quedo', 'pendiente', 'repara'), 939),
 (('transformador', 'quedo', 'pendiente'), 638),
 (('quedo', 'pendiente', 'reparar'), 488),
 (('normal', 'quedo', 'pendiente'), 487),
 (('quedo', 'pendiente', 'poste'), 449),
 (('quedo', 'pendiente', 'poda'), 388),
 (('quedo', 'pendiente', 'cuadrilla'), 334),
 (('quedo', 'pendiente', 'vehiculo'), 305),
 (('quedo', 'pendiente', 'realizar'), 252),
 (('quedo', 'normal', 'quedo'), 248),
 (('fusible', 'transformador', 'quedo'), 244),
 (('quedo', 'pendiente', 'transformador'), 216),
 (('quedo', 'normal', 'pendiente'), 205),
 (('pendiente', 'realizar', 'poda'), 204),
 (('fusible', 'quedo', 'pendiente'), 175),
 (('pendiente', 'poste', 'secundario'), 171),
 (('ruta', 'quedo', 'pendiente'), 171),
 (('linea', 'secundaria', 'rota'), 160),
 (('pendiente', 'reparar', 'linea'), 156),
 (('quedo', 'pendiente', 'caja'), 155)]